In [ ]:
%%HTML
<p id = 'heading1_p'> Google Cloud Connect - ATECC608B-TNGTLS </p>
<hr id = 'hline'> </hr> 
<p id = 'heading1_p' style='font-size: 20px; font-weight: bold;'>Introduction</p>
<div id='div_element'>
<p>Transport Layer Security 1.2 (TLS 1.2) has become the de facto standard for connecting embedded systems to a network. While TLS 1.2 is undeniably robust, an embedded system still requires a unique, secure and trusted identity to prevent large-scale remote attacks. </p>
<p>Securing communication with a Cloud service and manipulating keys comes with many challenges: storing and using keys in the microcontroller exposes them, operating systems and software have bugs, the Heartbleed bug for OpenSSL was notable by easily exposing keys. When combined with the TLS 1.2 protocol, Microchip’s ATECC608B Crypto Authentication device offers a unique, trusted, and verifiable identity that can help protect billions of connected devices by physically isolating keys and secrets from the application.​</p>
<p>This use case demonstrates Google cloud services.​</p>
</div>

<button class="collapsible" style='font-size: 20px; font-weight: bold;'>Cryptographic Asset list & Use Case implementation</button>
<div class="content">
<div id='content_element'>
<hr id = 'h_content'> </hr> 
<p style="float: left; font-size: 9pt; text-align: center; width: 30%; margin-right: 1%;">
    <img src="assets/images/assets_google.png" alt="Cryptographic Assets" style="height: 35em; width: 100%"><b>Cryptographic Asset list</b></p>
<p style="float: left; font-size: 9pt; text-align: center; 20em; width: 50%; margin-right: 1%;">
    <img src="assets/images/steps_google.png" alt="Use Case Steps" style="height: 25em; width: 100%"><b>Use Case implementation</b></p>
<p style="clear: both;">
</div>
</div>

<p id = 'heading1_p' style='margin-top:40px; font-size: 20px; font-weight: bold;'>Use Case information</p>
<hr id = 'hline'> </hr> 
<div id='div_element'>
<p> The Transaction Diagram represent all steps to be performed to successfully implement the given Use Case, 
during development and after product delivery to the end-user. </p>
<p><strong> For the ease of the execution, when steps are executed from transaction diagram, these are emulated in the Host-PC 
rather than executed in the Host-Microcontroller/Processor from the evaluation kit.</strong> The MPLAB-X project/C Source code includes software code to be executed in your Host-Microcontroller/Processor 
in your embedded project. </p>

<p>Select your Evaluation kit and then click on each step to execute the Use Case. Click on Usecase Help button in 
transaction diagram for additional details.</p>
</div>

In [ ]:
import os, shutil
from tpds.usecase_diagram import ProtoBoard, UsecaseDiagram
from tpds.tp_utils.tp_settings import TPSettings
from tpds.proto_boards import get_board_path
from IPython.display import display

curr_path = os.path.abspath('')
os.sys.path.extend([os.path.join(curr_path, 'assets', 'python')])
import google_connect

tp_settings = TPSettings()
usecase_working_dir = os.path.join(tp_settings.get_base_folder(), 'spg_cloud_connect')
os.makedirs(usecase_working_dir, exist_ok=True)
base_path = os.path.realpath(os.path.join(curr_path, '..'))
boards = ProtoBoard(max_width=400)
boards.add_board(
            'DM320118', 
            os.path.join(get_board_path(), 'DM320118', 'DM320118_WINC.png'), 
            os.path.join(get_board_path(), 'DM320118', 'DM320118.hex'),
            # Combined project for TrustFLEX and TNG cloud connect
            os.path.join(base_path, 'TrustFlex', 'firmware', 'cloud_connect_dm320118.X'))
boards.render_boards()

target_winc_path = os.path.join(tp_settings.get_base_folder(), 'winc_firmware_upgrade')
if not os.path.exists(target_winc_path):
    shutil.copytree(os.path.join(get_board_path(), 'DM320118', 'winc_firmware_upgrade'), target_winc_path)

google_connection = google_connect.GoogleConnectUsecase(boards)
usecase_td = UsecaseDiagram(os.path.join(curr_path, 'assets', 'images', 'td_google_connect.png'), working_dir=usecase_working_dir)

generate_manifest = usecase_td.add_script(236,69,35,30, google_connection.generate_or_upload_manifest)
register_certs = usecase_td.add_script(236,116,35,30, google_connection.register_device, prereq_scripts = [generate_manifest])
verify_cert_chain = usecase_td.add_script(242,222,35,30, google_connection.verify_cert_chain, prereq_scripts = [generate_manifest])
generate_jwt = usecase_td.add_script(501,281,35,30, google_connection.generate_JWT, prereq_scripts = [verify_cert_chain])
verify_with_rand_challenge = usecase_td.add_script(241,355,35,30, google_connection.verify_SE_with_random_challenge, prereq_scripts = [generate_jwt])

usecase_td.add_firmware(boards.get_firmware_project, tp_settings.get_mplab_paths().get('ide_path'))
usecase_td.add_image(718,214,140,30, os.path.join(curr_path, 'assets', 'images', 'c-snippet.png'), 'Code Snippet')
usecase_td.add_link(750,255,73,8,
    'https://github.com/MicrochipTech/cryptoauthlib/blob/af8187776cd3f3faf8bed412eaf6ff7221862e19/lib/jwt/atca_jwt.c#L64')
usecase_td.add_link(722,265,159,8,
    'https://github.com/MicrochipTech/cryptoauthlib/blob/af8187776cd3f3faf8bed412eaf6ff7221862e19/lib/jwt/atca_jwt.c#L251')
usecase_td.add_link(734,275,99,10,
    'https://github.com/MicrochipTech/cryptoauthlib/blob/af8187776cd3f3faf8bed412eaf6ff7221862e19/lib/jwt/atca_jwt.c#L105')
# Combined project for TrustFLEX and TNG cloud connect
usecase_td.add_usecase_help(os.path.join(base_path, 'TrustFlex', 'assets', 'Google Connect.md'))

display(boards.canvas)
usecase_td.display_canvas()